In [1]:
#!pip install cellpose imageio matplotlib numpy opencv-python scikit-image

from pathlib import Path
import pandas as pd
import numpy as np
import re
import os

from skimage.io import imread
from skimage.exposure import rescale_intensity
from skimage.morphology import disk, remove_small_objects
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops_table
from skimage.filters.rank import median
from cellpose import models
from tqdm import tqdm

In [2]:
# Initialize Cellpose models
model_nuclei = models.Cellpose(model_type='nuclei', gpu=False)
model_cyto3 = models.Cellpose(model_type='cyto3', gpu=False)

# Base path
BASE_PATH = Path("/Users/giselemiranda/MirandaLab/Collaborations/Marcelo-Unicamp/workshop-unicamp/data_day2/JUMP-CP_mini")
CLASSES = ['Brefeldin-A-like', 'Etoposide', 'Nocodazole', 'Rapamycin-like', 'Staurosporine-like']

# Function to load image
def load_image(path):
    return rescale_intensity(imread(str(path)), out_range=(0, 255)).astype(np.uint8)

def load_and_preprocess_mito_image(path):
    img = rescale_intensity(imread(str(path)), out_range=(0, 255)).astype(np.uint8)
    img = median(img, disk(2))
    #img = (equalize_adapthist(img, clip_limit=0.01) * 255).astype(np.uint8)
    img = rescale_intensity(img, out_range=(0, 255)).astype(np.uint8)

    return img

def ensure_segmentation_folder(image_path, prefix, diameter, flow_threshold, cellprob_threshold):
    seg_name = f"segmentation_{prefix}_d{diameter}_f{flow_threshold}_cp{cellprob_threshold}"
    seg_dir = image_path.parent / seg_name
    seg_dir.mkdir(exist_ok=True)
    return seg_dir / image_path.name

# Save mask
def save_mask(mask, out_path):
    from imageio import imwrite
    imwrite(out_path, (mask.astype(np.uint16)))

# Extract regionprops
#def get_props(mask, filename, cls, properties=["label", "area", "eccentricity", "solidity", "perimeter", "major_axis_length", "minor_axis_length"]):
#    lbl = label(mask)
#    props = regionprops_table(lbl, properties=properties)
#    df = pd.DataFrame(props)
#    df["filename"] = filename.name
#    df["class"] = cls
#    return df

def get_props(mask, intensity_img, filename, cls):
    lbl = label(mask)
    props = regionprops_table(
        lbl,
        intensity_image=intensity_img
        #properties=None  # this requests ALL supported properties
    )
    df = pd.DataFrame(props)
    df["filename"] = filename.name
    df["class"] = cls
    return df


# Parse files for 4 classes (standard naming)
def parse_standard_class(folder):
    files_by_id = {}
    subdirs = ["DNA", "Mito", "ER", "AGP"]
    pattern = re.compile(r'^(?P<id>CP\d-SC\d-\d+_[A-Z]\d{2}_T\d{4}F\d{3})[^C]*C(?P<ch>\d{2})')

    for subdir in subdirs:
        files = list((folder / "Orig" / subdir).glob("*.tif"))
        for f in files:
            m = pattern.match(f.name)
            if m:
                base_id = m.group("id")
                ch = m.group("ch")
                if base_id not in files_by_id:
                    files_by_id[base_id] = {}
                files_by_id[base_id][subdir] = f
    return files_by_id

# Parse files for Staurosporine-like (custom naming)
def parse_staurosporine_class(folder):
    files_by_id = {}
    channel_map = {
        "1": "Mito",
        "2": "AGP",
        "3": "RNA",
        "4": "ER",
        "5": "DNA"
    }

    all_files = list((folder / "Orig").rglob("*.tif")) + list((folder / "Orig").rglob("*.tiff"))

    pattern = re.compile(r'(?P<id>r\d{2}c\d{2}f\d{2}p\d{2})-ch(?P<ch>\d)')

    for f in all_files:
        match = pattern.search(f.name)
        if match:
            base_id = match.group("id")
            ch = match.group("ch")
            label = channel_map.get(ch)
            if label:
                if base_id not in files_by_id:
                    files_by_id[base_id] = {}
                files_by_id[base_id][label] = f
    return files_by_id

def segment_nuclei(dna_img, diameter=30, flow_threshold=0.4, cellprob_threshold=0.0, channels=[0, 0]):
    mask, _, _, _ = model_nuclei.eval(
        dna_img,
        diameter=diameter,
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold,
        channels=channels
    )
    return mask

def segment_cells(agp_img, er_img, dna_img, diameter=45, flow_threshold=0.4, cellprob_threshold=0.0, channels=[0, 1]):
    merged = np.stack([agp_img, er_img, dna_img], axis=-1)
    mask, _, _, _ = model_cyto3.eval(
        merged,
        diameter=diameter,
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold,
        channels=channels
    )
    return mask

def segment_mito(mito_img):
    if mito_img.max() == mito_img.min():
        # Avoid Otsu error on flat image
        return np.zeros_like(mito_img, dtype=np.uint8)
    
    thresh = threshold_otsu(mito_img)
    mask = (mito_img > thresh).astype(np.uint8)
    return mask



In [3]:
# Process all classes
all_nuclei, all_cells, all_mito = [], [], []

for cls in CLASSES:
    print(cls)
    folder = BASE_PATH / cls 
    if cls == "Staurosporine-like":
        matched = parse_staurosporine_class(folder)
    else:
        matched = parse_standard_class(folder)

    #for img_id, channels in matched.items():
    for img_id, channels in tqdm(matched.items(), desc=f"Processing {cls}"):
        #class_name = cls
        dna_path = channels.get("DNA")
        agp_path = channels.get("AGP")
        er_path  = channels.get("ER")
        mito_path = channels.get("Mito")
    
        # Skip if critical images are missing
        if not (dna_path and agp_path and er_path and mito_path):
            continue
    
        # --- Load images ---
        dna = load_and_preprocess_mito_image(dna_path)
        agp = load_and_preprocess_mito_image(agp_path)
        er  = load_and_preprocess_mito_image(er_path)
        mito = load_and_preprocess_mito_image(mito_path)
    
        # --- Segment nuclei ---
        mask_nuclei = segment_nuclei(dna, diameter=20, flow_threshold=0.4, cellprob_threshold=-2)
        out_nuclei = ensure_segmentation_folder(dna_path, "nuclei", 20, 0.4, -2.0)
        save_mask(mask_nuclei, out_nuclei)
        all_nuclei.append(get_props(mask_nuclei, dna, dna_path, cls))
    
        # --- Segment cells ---
        cells_merged = np.stack([agp, er], axis=-1)
        mask_cells = segment_cells(agp, er, dna, diameter=50, flow_threshold=0.5, cellprob_threshold=-4)
        out_cells = ensure_segmentation_folder(agp_path, "cells", 50, 0.5, -4.0)
        save_mask(mask_cells, out_cells)
        all_cells.append(get_props(mask_cells, cells_merged, agp_path, cls))
    
        # --- Segment mito (Otsu) ---
        mask_mito = segment_mito(mito)
        out_mito = ensure_segmentation_folder(mito_path, "mito", 0, 0, 0)
        save_mask(mask_mito, out_mito)
        all_mito.append(get_props(mask_mito, mito, mito_path, cls))

# Concatenate all results
df_nuclei = pd.concat(all_nuclei, ignore_index=True)
df_cells = pd.concat(all_cells, ignore_index=True)
df_mito = pd.concat(all_mito, ignore_index=True)

# Show summaries
#import ace_tools as tools; tools.display_dataframe_to_user(name="Nuclei Regionprops", dataframe=df_nuclei)
#tools.display_dataframe_to_user(name="Cell Body Regionprops", dataframe=df_cells)
#tools.display_dataframe_to_user(name="Mito Regionprops", dataframe=df_mito)


Brefeldin-A-like


Processing Brefeldin-A-like: 100%|████████████████| 3/3 [00:26<00:00,  8.71s/it]


Etoposide


Processing Etoposide: 100%|███████████████████████| 3/3 [00:26<00:00,  8.88s/it]


Nocodazole


Processing Nocodazole: 100%|██████████████████████| 3/3 [00:22<00:00,  7.38s/it]


Rapamycin-like


Processing Rapamycin-like: 100%|██████████████████| 3/3 [00:42<00:00, 14.06s/it]


Staurosporine-like


Processing Staurosporine-like: 100%|██████████████| 3/3 [00:12<00:00,  4.11s/it]


In [4]:
df_nuclei

,label,bbox-0,bbox-1,bbox-2,bbox-3,filename,class
0,1,0,0,13,14,CP1-SC1-08_P06_T0001F003L01A01Z01C01.tif,Brefeldin-A-like
1,2,0,259,10,280,CP1-SC1-08_P06_T0001F003L01A01Z01C01.tif,Brefeldin-A-like
2,3,0,549,14,581,CP1-SC1-08_P06_T0001F003L01A01Z01C01.tif,Brefeldin-A-like
3,4,0,672,25,699,CP1-SC1-08_P06_T0001F003L01A01Z01C01.tif,Brefeldin-A-like
4,5,0,1054,23,1077,CP1-SC1-08_P06_T0001F003L01A01Z01C01.tif,Brefeldin-A-like
...,...,...,...,...,...,...,...
1395,243,1043,550,1072,600,CP1-SC1-01_G09_T0001F001L01A01Z01C01.tif,Rapamycin-like
1396,244,1048,615,1073,657,CP1-SC1-01_G09_T0001F001L01A01Z01C01.tif,Rapamycin-like
1397,245,1051,444,1080,472,CP1-SC1-01_G09_T0001F001L01A01Z01C01.tif,Rapamycin-like
1398,246,1070,695,1080,724,CP1-SC1-01_G09_T0001F001L01A01Z01C01.tif,Rapamycin-like
